# Data Engineering

In [ ]:
#create environment
import numpy as np
import pandas as pd
import scipy as sci
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from copy import deepcopy
from statsmodels.formula.api import ols
import statsmodels.api as sm
from datetime import datetime
from pandas.plotting import scatter_matrix
pd.set_option('display.max_columns', 500)

### Data

In [ ]:
#import data
uno_sale = pd.read_pickle('uno_sale_ec.pkl')
uno_inv = pd.read_pickle('uno_inv_ec.pkl')

In [ ]:
invqty = uno_inv.groupby(['STYLE#','COLOR']).agg({'Description':'count','Qty':'sum', 'On-Hand':'sum'}).sort_values('Qty', ascending=False).reset_index()
invqty['Warehouse'] = invqty['On-Hand'] /invqty['Description']
invqty.drop(columns=['Description','On-Hand'], inplace=True)
invqty['Discrepancy'] = invqty.Warehouse - invqty.Qty
invqty.head()

In [ ]:
invqty['Warehouse'] = invqty['Qty']
invqty.drop(columns=['Discrepancy','Qty'], inplace=True)
invqty.head()

In [ ]:
uno_inv.drop(columns=['Categories','On-Hand'],inplace=True)
uno_inv.head()

In [94]:
uno_inv['Sale_Qty'] = np.abs([x if x < 0 else 0 for x in uno_inv.Qty])
uno_inv['Inv_Qty'] = np.abs([x if x > 0 else 0 for x in uno_inv.Qty])
uno_inv.head()

,STYLE#,COLOR,Description,LAST RCVD,Qty,Sale_Qty,Inv_Qty
0,DTA1A9500,IV,seed bead anklet,2016-04-12,-144.0,144.0,0.0
1,DTA1A9500,IV,seed bead anklet,2016-04-12,144.0,0.0,144.0
2,DTA1A9500,MUL,seed bead anklet,2016-04-12,-144.0,144.0,0.0
3,DTA1A9500,MUL,seed bead anklet,2016-04-12,144.0,0.0,144.0
4,DTA1A9500,TQ,seed bead anklet,2016-04-12,-144.0,144.0,0.0


In [95]:
uno_inv_agg = uno_inv.groupby(['STYLE#','COLOR','LAST RCVD']).agg('sum').reset_index()
uno_inv_agg.head()

,STYLE#,COLOR,LAST RCVD,Qty,Sale_Qty,Inv_Qty
0,DTA1A9500,IV,2016-04-12,0.0,144.0,144.0
1,DTA1A9500,MUL,2016-04-12,0.0,144.0,144.0
2,DTA1A9500,TQ,2016-04-12,0.0,144.0,144.0
3,DTA1B2029,WG-COR,2016-04-04,108.0,12.0,120.0
4,DTA1B2029,WG-COR,2016-04-06,-12.0,12.0,0.0


In [98]:
uno_inv_agg = pd.merge(uno_inv_agg, invqty, how='left', left_on=['STYLE#','COLOR'],right_on=['STYLE#','COLOR'])
uno_inv_agg.head()

,STYLE#,COLOR,LAST RCVD_x,Qty_x,Sale_Qty_x,Inv_Qty_x,Warehouse_x,Description,LAST RCVD_y,Qty_y,Sale_Qty_y,Inv_Qty_y,Warehouse_y
0,DTA1A9500,IV,2016-04-12,0.0,144.0,144.0,0.0,seed bead anklet,2016-04-12,-144.0,144.0,0.0,0.0
1,DTA1A9500,IV,2016-04-12,0.0,144.0,144.0,0.0,seed bead anklet,2016-04-12,144.0,0.0,144.0,0.0
2,DTA1A9500,MUL,2016-04-12,0.0,144.0,144.0,0.0,seed bead anklet,2016-04-12,-144.0,144.0,0.0,0.0
3,DTA1A9500,MUL,2016-04-12,0.0,144.0,144.0,0.0,seed bead anklet,2016-04-12,144.0,0.0,144.0,0.0
4,DTA1A9500,TQ,2016-04-12,0.0,144.0,144.0,0.0,seed bead anklet,2016-04-12,-144.0,144.0,0.0,0.0


In [ ]:
uno_inv_subset = uno_inv.drop(columns=['LAST RCVD', 'Qty','Sale_Qty','Inv_Qty'])
uno_inv_agg = pd.merge(uno_inv_agg, uno_inv_subset,\
                       how='left', left_on=['STYLE#','COLOR'],right_on=['STYLE#','COLOR'])

uno_inv_agg.head(30)